# OSM PBF Loader

`OSMPbfLoader` can really quickly parse full OSM extract in the form of `*.osm.pbf` file.

It can download and parse a lot of features much faster than the `OSMOnlineLoader`, but it's much more useful when a lot of different features are required at once (like when using predefined filters).

When only a single or few features are needed, `OSMOnlineLoader` might be a better choice, since `OSMPbfLoader` will use a full extract of all features in a given region and will have to iterate over all of them.

In [ ]:
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER, GEOFABRIK_LAYERS
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from srai.loaders.osm_loaders import OSMPbfLoader
from srai.constants import REGIONS_INDEX, WGS84_CRS
from srai.regionalizers import geocode_to_region_gdf
from srai.geometry import buffer_geometry

from shapely.geometry import Point, box
import geopandas as gpd

## Using OSMPbfLoader to download data for a specific area

### Download all features from `HEX2VEC_FILTER` in Warsaw, Poland

In [ ]:
loader = OSMPbfLoader()
warsaw_gdf = geocode_to_region_gdf("Warsaw, Poland")
warsaw_features_gdf = loader.load(warsaw_gdf, HEX2VEC_FILTER)
warsaw_features_gdf

### Plot features

Inspired by [`prettymaps`](https://github.com/marceloprates/prettymaps)

In [ ]:
clipped_features_gdf = warsaw_features_gdf.clip(warsaw_gdf.geometry.unary_union)

In [ ]:
ax = warsaw_gdf.plot(color="lavender", figsize=(16, 16))

# plot water
clipped_features_gdf.dropna(subset=["water", "waterway"], how="all").plot(
    ax=ax, color="deepskyblue"
)

# plot greenery
clipped_features_gdf[
    clipped_features_gdf["landuse"].isin(
        ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"]
    )
].plot(ax=ax, color="mediumseagreen")

# plot buildings
clipped_features_gdf.dropna(subset=["building"], how="all").plot(
    ax=ax, color="dimgray", markersize=0.1
)

xmin, ymin, xmax, ymax = warsaw_gdf.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()

### Download all features from popular tags based on OSMTagInfo in Vienna, Austria

In [ ]:
popular_tags = get_popular_tags(in_wiki_only=True)

num_keys = len(popular_tags)
f"Unique keys: {num_keys}."

In [ ]:
{k: popular_tags[k] for k in list(popular_tags)[:10]}

In [ ]:
vienna_center_circle = buffer_geometry(Point(16.37009, 48.20931), meters=1000)
vienna_center_circle_gdf = gpd.GeoDataFrame(
    geometry=[vienna_center_circle],
    crs=WGS84_CRS,
    index=gpd.pd.Index(data=["Vienna"], name=REGIONS_INDEX),
)

In [ ]:
loader = OSMPbfLoader()
vienna_features_gdf = loader.load(vienna_center_circle_gdf, popular_tags)
vienna_features_gdf

### Plot features

Uses `default` preset colours from [`prettymaps`](https://github.com/marceloprates/prettymaps)

In [ ]:
clipped_vienna_features_gdf = vienna_features_gdf.clip(vienna_center_circle)

In [ ]:
ax = vienna_center_circle_gdf.plot(color="#F2F4CB", figsize=(16, 16))

# plot water
clipped_vienna_features_gdf.dropna(subset=["water", "waterway"], how="all").plot(
    ax=ax, color="#a8e1e6"
)

# plot streets
clipped_vienna_features_gdf.dropna(subset=["highway"], how="all").plot(
    ax=ax, color="#475657", markersize=0.1
)

# plot buildings
clipped_vienna_features_gdf.dropna(subset=["building"], how="all").plot(ax=ax, color="#FF5E5B")

# plot parkings
clipped_vienna_features_gdf[
    (clipped_vienna_features_gdf["amenity"] == "parking")
    | (clipped_vienna_features_gdf["highway"] == "pedestrian")
].plot(ax=ax, color="#2F3737", markersize=0.1)

# plot greenery
clipped_vienna_features_gdf[
    clipped_vienna_features_gdf["landuse"].isin(
        ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"]
    )
].plot(ax=ax, color="#8BB174")

xmin, ymin, xmax, ymax = vienna_center_circle_gdf.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()

### Download all grouped features based on Geofabrik layers in New York, USA

In [ ]:
manhattan_bbox = box(-73.994551, 40.762396, -73.936872, 40.804239)
manhattan_bbox_gdf = gpd.GeoDataFrame(
    geometry=[manhattan_bbox],
    crs=WGS84_CRS,
    index=gpd.pd.Index(data=["New York"], name=REGIONS_INDEX),
)

In [ ]:
loader = OSMPbfLoader()
new_york_features_gdf = loader.load(manhattan_bbox_gdf, GEOFABRIK_LAYERS)
new_york_features_gdf

### Plot features

Inspired by https://snazzymaps.com/style/14889/flat-pale

In [ ]:
ax = manhattan_bbox_gdf.plot(color="#e7e7df", figsize=(16, 16))

# plot greenery
new_york_features_gdf[new_york_features_gdf["leisure"] == "leisure=park"].plot(
    ax=ax, color="#bae5ce"
)

# plot water
new_york_features_gdf.dropna(subset=["water", "waterways"], how="all").plot(ax=ax, color="#c7eced")

# plot streets
new_york_features_gdf.dropna(subset=["paths_unsuitable_for_cars"], how="all").plot(
    ax=ax, color="#e7e7df", linewidth=1
)
new_york_features_gdf.dropna(
    subset=["very_small_roads", "highway_links", "minor_roads"], how="all"
).plot(ax=ax, color="#fff", linewidth=2)
new_york_features_gdf.dropna(subset=["major_roads"], how="all").plot(
    ax=ax, color="#fac9a9", linewidth=3
)

# plot buildings
new_york_features_gdf.dropna(subset=["buildings"], how="all").plot(ax=ax, color="#cecebd")

xmin, ymin, xmax, ymax = manhattan_bbox_gdf.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()

## Using OSMPbfLoader to download data for a specific area and transforming it to GeoParquet file

### Download all grouped features based on Geofabrik layers in Reykjavík, Iceland

In [ ]:
loader = OSMPbfLoader()
reykjavik_gdf = geocode_to_region_gdf("Reykjavík, IS")
reykjavik_features_gpq = loader.load_to_geoparquet(reykjavik_gdf, GEOFABRIK_LAYERS)
reykjavik_features_gpq

### Read those features using DuckDB

In [ ]:
import duckdb

connection = duckdb.connect()

connection.load_extension("parquet")
connection.load_extension("spatial")

features_relation = connection.read_parquet([str(path) for path in reykjavik_features_gpq]).project(
    "* REPLACE (ST_GeomFromWKB(geometry) AS geometry)"
)
features_relation

### Count all buildings

In [ ]:
features_relation.filter("buildings IS NOT NULL").count("feature_id")

### Download main roads for Estonia using `PbfFileHandler`

`PbfFileHandler` is a special class dedicated for reading `*.osm.pbf` files and is used by `OSMPbfLoader` to load features.
It allows for parsing `pbf` files without any tags or geometry filtering, that is automatically applied by `OSMPbfLoader`.

In [ ]:
highways_filter = {
    "highway": [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
    ]
}

In [ ]:
from srai.loaders import download_file
from srai.loaders.osm_loaders.pbf_file_handler import PbfFileHandler

estonia_pbf_url = "http://download.geofabrik.de/europe/estonia-latest.osm.pbf"
estonia_pbf_file = "estonia-latest.osm.pbf"
download_file(estonia_pbf_url, estonia_pbf_file, force_download=False)

handler = PbfFileHandler(
    geometry_filter=None, tags_filter=highways_filter
)  # parsing pbf file without tags filtering requires a lot of memory available in the system
estonia_features_gpq = handler.convert_pbf_to_gpq(estonia_pbf_file)
estonia_features_gpq

In [ ]:
features_relation = connection.read_parquet(str(estonia_features_gpq)).project(
    "* REPLACE (ST_GeomFromWKB(geometry) AS geometry)"
)
features_relation

### Count loaded roads

In [ ]:
features_relation.count("feature_id")

### Calculate roads length
We will transform the geometries to the Estonian CRS - [EPSG:3301](https://epsg.io/3301)

In [ ]:
length_in_meters = (
    features_relation.project(
        "ST_Length(ST_Transform(geometry, 'EPSG:4326', 'EPSG:3301')) AS road_length"
    )
    .sum("road_length")
    .fetchone()[0]
)
length_in_km = length_in_meters / 1000
length_in_km

### Plot the roads using GeoPandas

With fast loading of geoparuqet files using `geoarrow.pyarrow` library.

In [ ]:
import geoarrow.pyarrow as ga
from geoarrow.pyarrow import io

from srai.constants import GEOMETRY_COLUMN

parquet_table = io.read_geoparquet_table(estonia_features_gpq)
ga.to_geopandas(parquet_table.column(GEOMETRY_COLUMN)).plot()